In [ ]:
!ls /content/drive/MyDrive/ECE_542/Competition_Project

data.zip  metadata  ml_utils  test_data.zip


In [ ]:
!cp /content/drive/MyDrive/ECE_542/Competition_Project/test_data.zip .

In [ ]:
!unzip test_data.zip

Archive:  test_data.zip
   creating: test_data/
   creating: test_data/preprocessed_data/
  inflating: test_data/preprocessed_data/subject_012_session_01__x.csv  
  inflating: test_data/preprocessed_data/subject_010_session_01__y.csv  
  inflating: test_data/preprocessed_data/subject_012_session_01__y.csv  
  inflating: test_data/preprocessed_data/subject_011_session_01__x.csv  
  inflating: test_data/preprocessed_data/subject_010_session_01__x.csv  
  inflating: test_data/preprocessed_data/subject_009_session_01__y.csv  
  inflating: test_data/preprocessed_data/subject_011_session_01__y.csv  
  inflating: test_data/preprocessed_data/subject_009_session_01__x.csv  


In [ ]:
!rm test_data.zip

In [ ]:
!cp -r /content/drive/MyDrive/ECE_542/Competition_Project/ml_utils .
!ls

drive  ml_utils  sample_data  test_data


In [ ]:
base_path = "/content/drive/MyDrive/ECE_542/Competition_Project"

In [ ]:
!ls /content/drive/MyDrive/ECE_542/Competition_Project

ls: cannot access '/content/drive/MyDrive/ECE_542/Competition_Project': No such file or directory


In [ ]:
import os
import sys

In [ ]:
import json
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np
from ml_utils.dataset import SubjectDataset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [ ]:
!ls test_data/preprocessed_data/

subject_009_session_01__x.csv  subject_011_session_01__x.csv
subject_009_session_01__y.csv  subject_011_session_01__y.csv
subject_010_session_01__x.csv  subject_012_session_01__x.csv
subject_010_session_01__y.csv  subject_012_session_01__y.csv


In [ ]:
#testing data path
test_data_path = os.path.join("test_data", "preprocessed_data")

In [ ]:
class OneDConvNet(nn.Module):
  def __init__(self, n_features, n_classes):
    super(OneDConvNet, self).__init__()
    self.Convolution_Layer_1 = nn.Conv1d(in_channels=n_features, out_channels=8, kernel_size=3, stride=1)
    self.Pool_Layer1         = nn.MaxPool1d(kernel_size=2, stride=2)
    self.Convolution_Layer_2 = nn.Conv1d(in_channels= 8, out_channels=16, kernel_size=3, stride=1)
    self.Pool_Layer2         = nn.MaxPool1d(kernel_size=2, stride=2)
    self.Convolution_Layer_3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
    self.Pool_Layer3         = nn.MaxPool1d(kernel_size=2, stride=2)
    self.FullConnected1      = nn.Linear(32*3,16)
    self.FullConnected2      = nn.Linear(16,8)
    self.FullConnected3      = nn.Linear(8, n_classes)
  
  def forward(self, x):
    x = self.Pool_Layer1(F.relu(self.Convolution_Layer_1(x)))
    x = self.Pool_Layer2(F.relu(self.Convolution_Layer_2(x)))
    x = self.Pool_Layer3(F.relu(self.Convolution_Layer_3(x)))
    N, C, T = x.size()
    x = x.view(-1, C*T) # Flatten
    x = F.relu(self.FullConnected1(x))
    x = F.relu(self.FullConnected2(x))
    x = self.FullConnected3(x)
    return x

In [ ]:
best_model = OneDConvNet(6, 4)
best_model.load_state_dict(torch.load(os.path.join(base_path, "Models",  "model.pth")))
best_model = best_model.to(device)

In [ ]:
import pandas as pd

In [ ]:
#main_cell
split_ids = ["009_01", "010_01", "011_01", "012_01"]
batch_size = 256
for id in split_ids:

    test_dataset = SubjectDataset(
      test_data_path, 
      [id], 
      cache_len = 1
    )
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False)

    output = []
    #labels = []

    for X, y in test_dataloader:

        X = X.float().to(device)
        y = y.view(X.size(0)).to(device)

        y_pred = best_model(X)
        predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()

        output.append(predicted_classes)

    _output = np.concatenate(output, axis=0)

    df = pd.DataFrame({"label": _output})

    subject_id, session_id = id.split("_")

    filename = "subject_{}_{}__y.csv".format(subject_id, session_id)
    df.to_csv(f"/content/drive/MyDrive/ECE_542/Competition_Project/Predictions/{filename}", header=False, index=False)